The purpose of this notebook is to indentify what cells were used for testing.

In [14]:
# open test_results.pkl

import pickle
import numpy as np
from core.backend import open_minian

with open('test_results.pkl', 'rb') as f:
    results = pickle.load(f)

In [9]:
# Iterate through experiments
dict = {"cross_session_same_day": [["./data/PL010/PL010_D1S1", "./data/PL010/PL010_D1S4"], ["./data/AA058/AA058_D1S1", "./data/AA058/AA058_D1S4"], ["./data/AA036/AA036_D2S1", "./data/AA036/AA036_D2S4"], ["./data/AA034/AA034_D1S1", "./data/AA034/AA034_D1S4"]],
        "cross_day_same_session": [["./data/PL010/PL010_D1S1", "./data/PL010/PL010_D8S1"], ["./data/AA058/AA058_D1S1", "./data/AA058/AA058_D5S1"], ["./data/AA036/AA036_D2S1", "./data/AA036/AA036_D6S1"], ["./data/AA034/AA034_D1S1", "./data/AA034/AA034_D7S1"]],
        "cross_day_cross_session": [["./data/PL010/PL010_D1S1", "./data/PL010/PL010_D8S4"], ["./data/AA058/AA058_D1S1", "./data/AA058/AA058_D5S4"], ["./data/AA036/AA036_D2S1", "./data/AA036/AA036_D6S4"], ["./data/AA034/AA034_D1S1", "./data/AA034/AA034_D7S4"]],
        "cross_animal": [["./data/PL010/PL010_D1S1", "./data/AA058/AA058_D1S1"], ["./data/AA058/AA058_D1S1", "./data/AA036/AA036_D2S1"], ["./data/AA036/AA036_D2S1", "./data/AA034/AA034_D1S1"], ["./data/AA034/AA034_D1S1", "./data/PL010/PL010_D1S1"]],
        "within_session": ["./data/PL010/PL010_D1S1", "./data/PL010/PL010_D1S1", "./data/PL010/PL010_D1S1", "./data/PL010/PL010_D1S1"]}

experiments = results.keys()

def find_test_set(experiment, training_set):
        arr = dict[experiment]
        for train, test in arr:
            if train[-len(training_set):] == training_set:
                return test
        print(f"Error: Test set not found for {experiment} and {training_set}")

def get_ground_truth(arr):
        # Get rows 0, 2, 4 etc. from the array
        return arr[::2]
        

In [8]:
results["cross_session_same_day"]["AA034_D1S1"]

(60, 132310)

In [20]:
test_cells_dict = {}

for experiment in experiments:
    test_cells_dict[experiment] = {}
    training_sets = results[experiment].keys()
    for training_set in training_sets:
        test_cells_dict[experiment][training_set] = []
        test_set = find_test_set(experiment, training_set)
        E = open_minian(test_set)["E"].load()
        all_unit_ids = E.unit_id.values
        verified = E.verified.values.astype(int)
        unit_ids = all_unit_ids[verified==1]
        E = E.sel(unit_id=unit_ids)
        ground_truth = get_ground_truth(results[experiment][training_set])
        # We need to match the ground truth with the results in E and return the name of the cell
        for row in ground_truth:
            found = False
            for unit_id in unit_ids:
                if E.sel(unit_id=unit_id).values == row:
                    test_cells_dict[experiment][training_set].append(unit_id)
                    found = True
                    break
        
            if not found:
                raise Exception(f"Error: Cell not found in test set {test_set} for experiment {experiment} and training set {training_set}")


        
    
    